# Creating Customer Segments

**Project Status: In Progress (Updated: 3/14/2016)

This project will analyze a dataset containing annual spending amounts for internal structure, to understand the variation in the different types of customers that a wholesale distributor interacts with.

In [ ]:
# Import libraries: NumPy, pandas, matplotlib
import numpy as np
import pandas as pd
import scipy 
import matplotlib.pyplot as pl
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA, FastICA
from sklearn import preprocessing
from IPython.display import HTML

%matplotlib inline

In [ ]:
# Read dataset
data = pd.read_csv("wholesale-customers.csv")
print "Dataset has {} rows, {} columns".format(*data.shape)

print "\nStandard Deviation:"
print data.std()

X=data.ix[:,0:6].values

In [ ]:
data.head()

**1)** Before doing any computations, what do you think will show up in your computations? List one or two ideas for what might show up as the first PCA dimensions, or what type of vectors will show up as ICA dimensions.

-  Principal Component Analysis (PCA) is utilized to identify patterns in dataset based on the correlation between the n-features. PCA attempts to find the directions of maximum variance in a high-dimensional data (d=6) and the projects it onto a new subspace (k<=d). We would expect the first several PCs to be composed of original attributes that are in some way correlated with each other. These principal components could represent the customers that purchase certain items or a combination of the items. Additionally, before running any PCA, plotting the histograms (shown below) seems to indicate a correlation in the number of orders among `Fresh`, `Milk`, and `Grocery`. Intuitively, the histograms show an exponential decline in the number of orders for the respected products, hence this could represent a cluster of the larger companies. In contrast to the remaining products, the histogram virtually drops off after the first two bins (lower number of orders) and perhaps indicate the smaller company purchases from the wholesale grocery distributor. In addition to `Fresh` and `Milk` having the highest standard deviations of 12,647 and 9,503 respectively, I would expect these two items to have a high significance (magnitude) in the eigenvector.


- Independent Component Analysis (ICA) is a statistical technique for identifying the underlying hidden factors for a given multidimensional dataset. In short, ICA produces dimensions of variation where the dimensions (features) are as statistically independent from one another. The data variables are assumed to be linear mixtures of some unknown "latent" variables and the mixing system is also unknown. The assumption is that the latent variables are both nongaussian and mutually independent; these variables are called the independent components. In the context of this project we are attempting to identify the different buyer types and cluster them together. Being statistically independent, the different types of buyers could mean the size of the purchaser (small/larger) or different consumption patterns. For example, there are buyers who mostly purchased Milk and Grocery, and less Detergents/Paper or Delicatessen. Likely this could represent the type of purchasers for a small market or convenience store. 


In [ ]:
data.plot(kind='hist',alpha=0.5,bins = 30, subplots=True, layout=(3,2), legend=True, figsize=(12,10))

In [ ]:
data.describe()

In [ ]:
data.corr()

In [ ]:
cov_matrix = data.cov()
cov_matrix

In [ ]:
#Eigenvalue and Eigenvector of covariance matrix
columns=data.columns
eig_vals, eig_vecs = np.linalg.eig(cov_matrix)
sum_ev = sum(eig_vals)
#Explained Variance Ratio
print eig_vals/sum_ev

# PCA

**2)** How quickly does the variance drop off by dimension? If you were to use PCA on this dataset, how many dimensions would you choose for your analysis? Why?

- The first PC accounts for the highest portion of the total variance or "spread" of the data. The subsequent PC's account for the remaining variability, and usually the first couple of PC's account for roughly 90-95% of the total variance. The below plot shows the variance vs the number of principal components (k=5). Based on this plot, the number of dimensions would be reduced to 2, which explains roughly 86% of the total variance. In this plot, the ['Explained Variance Ratio`](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) 
is a fraction of the total variance explained by each of the selected components. In regards to how quickly does the variance drop off, we can compute the slope of the number of PC components (x) vs the explained variance (y). The slope between PC 1-2 is -0.054 and the slope between 2-3 is -0.335. Furthermore, the slope between PC 3-4 is -0.026 and at this point, the slope begins to level out. Therefore between PC 2-3 you observe the steepest slope as the explained variance ratio begins to quickly decline. The bar plot below is an alternative visualization to observe the change in variance per PC. 
 
 
- As a note, for PCA to be implemented correctly, the dataset needs to be standardized (mean equal 0 and standard deviation =1. You need to address the event where features are on different scales or different magnitudes of range, and thus causing the results of PCA to be biased in the direction of the features with a larger range. One approach to scaling the data is using [`Scale`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html); centers the mean and component wise scale to unit variance (1). However, it may not always be the case in which the data should be standardized. In some instances you may want to preserve the variance of each dimension as opposed to scaling to unit length, which could remove the potential dependence between features since each feature is scaled independently. The features were not scaled to stdev = 1 for this dataset as the attributes have the same units monetary units (m.u.) and in essence, we are trying to capture the variance in spending among the different types of customers.

#### **3)** What do the dimensions seem to represent? How can you use this information? 

- In the original dimensional space the dimensions of the axes are in terms of the amount of monetary units (m.u.) the food distributor sales. The axes are then transposed into the new dimensional space and is now represented by the principal components. First, we must begin by constructing the covariance matrix (S); covariance is the measures how much the features vary from the mean with respect to each other. For this dataset, there are 440 elements (n=440) and 6 measurements (m=6), so the covariance matrix would be a 440x6 matrix; with 6 eigenvalues. The total variance is the Trace of the S, which is just the summation of the main diagonal. Next the covariance matrix is decomposed into its eigenvectors and eigenvalues; the eigenvectors of S represents the principal components (directions of maximum variance and determined the direction of the new feature space) and the eigenvalues (scalar) correspond to the magnitude of the eigenvectors; in other words, the eignevalues explains the variance along the new featue axes. The eigenvalues are sorted in descending order (largest first) and dividing each eigenvalue by the total variance yields the fraction of variance explained. We are only interested in the top eigenvectors based on the values of their corresponding eigenvalues. The eigenvector corresponding to the highest eigenvalue will be the first PC. The eigenvector (v1) points in the direction of maximum variance (most information gained) of the dataset. The second eigenvector (v2) will point in direction orthogonal (perpendicular in 2D) to v1. One of the main advantages of using PCA is if you have a dataset with high dimensions you can reduce the number dimensions without losing much of the information. Using feature extraction, the data is projected onto a new feature space and thus reducing the number dimensions through data compression; the goal is to maintain the most relevant information. Being that the objective is to reduce the dimensionality of the data by compressing it onto a new feature subspace, we chose only the subset of the eigenvectors (principal components) that contains most of the information (variance). 


- The dimensions of the first PC (explains the most variability in the data) and has coefficients of `Fresh = -0.976`, `Frozen = -0.152`,`Milk = -0.121`, etc. We can clearly see that fresh items have the largest magnitude, followed by frozen products, and then milk. The descriptive statistics shows that on average, fresh items are purchased 55% more than the other 5 products. The annual spending on fresh items has the largest standard deviation (12,647) which suggest there is a lot of variance between the customers on how much they spend on fresh items. As the first PC identifies the item as the most important factor among the features, it would be a good idea to increase marketing towards the customer base on fresh products as it seems to be a cluster of those who buy large quantities  of fresh products while there are customers who make smaller purchase orders. 


- Second Principal Component gives the following magnitudes for the eigenvector v2: `Fresh = -0.110614` `Milk =0.515802`, `Grocery =0.764606`, `Frozen = -0.018723` etc. Looking at the magnitude of the eigenvector, a correlation exist among `Grocery` and `Milk`; customers who buy more 'grocery' items also tend to buy more 'milk' items as well. Therefore, it would be advised to market grocery and milk products together given their correlation in order to increase sales. Bundling these two items when shipping to the customer will also reduce cost.

In [ ]:
# TODO: Apply PCA with the same number of dimensions as variables in the dataset

#Subtracting out the mean to center the data:
columns = data.columns
mean = data.mean()
df_m = pd.DataFrame(mean)
df_mean = df_m.transpose()
data_center = pd.DataFrame(data[columns].values - df_mean[columns].values, columns=columns)
#print data_center

def doPCA():
    pca = PCA(n_components=5)
    pca.fit(data_center)
    return pca

# Print the components and the amount of variance in the data contained in each dimension
pca = doPCA()
columns = data_center.columns
df_ica = pd.DataFrame(pca.components_, columns = columns, index=['1', '2', '3', '4','5'])
df_ica.index.names = ['PC']
print "Principal Component Analysis:"
print df_ica

ex_var = pca.explained_variance_ratio_
df_var = pd.Series(ex_var,index=['1','2','3','4','5'])
df_var.sort(ascending=False)


df_var.index.names = ['PC']
print "\nExplained Variance of Each Component:"
print df_var

In [ ]:
pl.figure(figsize=(8,5))
x = np.arange(1,6)
y = np.cumsum(ex_var)
pl.plot(x,y,marker ="o", mfc='#780000', color = '#CC0000')
pl.xlabel("No. of PC Components", fontsize = 14)
pl.ylabel("Cumulative Explained Variance Ratio", fontsize =14)
pl.title("No. of PC Components vs Explained Variance Ratio", fontsize = 16)
pl.show()

In [ ]:
pl.figure(figsize=(10,6))
pl.bar(range(1,6), ex_var, alpha = .8, align='center',
       label = 'Individual Explained Variance', color = 'blue')
pl.ylabel('Explained Variance Ratio', fontsize = 14)
pl.xlabel('Princial Components', fontsize = 14)
pl.title('Explained Variance', fontsize = 16)
pl.show

#Slope:
X1, Y1 = 1, 0.459614
X2, Y2 = 2, 0.405172
X3, Y3 = 3, 0.070030
X4, Y4 = 4, 0.044023
slope1 = (Y2-Y1)/(X2-X1)
slope2 = (Y3-Y2)/(X3-X2)
slope3= (Y4-Y3)/(X4-X3)
print "Slope1:", slope1
print "Slope2:", slope2
print "Slope3:", slope3

In [ ]:
def biplot(df):
    # Fit on 2 components
    #Scaling 
    pca = PCA(n_components=2, whiten=True).fit(df)
    
    # Plot transformed/projected data
    ax = pd.DataFrame(
        pca.transform(df),
        columns=['PC1', 'PC2']
    ).plot(kind='scatter', x='PC1', y='PC2', figsize=(8, 6), color = 'g', s=5)
    # Plot arrows and labels
    for i, (pc1, pc2) in enumerate(zip(pca.components_[0], pca.components_[1])):
        ax.arrow(0, 0, pc1, pc2, width=0.001, fc='r', ec='r')
        ax.annotate(df.columns[i], (pc1, pc2), size=12)
    return ax

ax = biplot(data)
pl.title("Biplot", fontsize = 20)
pl.xlabel("Principal Component: 1", fontsize = 14)
pl.ylabel("Principal Component: 2", fontsize = 14)
ax.set_xlim([-1.5, .5])
ax.set_ylim([-1.0, 1.5])

# ICA

4) For each vector in the ICA decomposition, write a sentence or two explaining what sort of object or property it corresponds to. What could these components be used for?

- [`Independent Component Analysis`](http://scikit-learn.org/stable/modules/decomposition.html#ica) is an algorithm that finds directions in the feature space corresponding to projections with high non-Gaussianity. ICA will provide six new vectors with each vector being an independent component. Furthermore with ICA, we can attempt to identify the different types of customers so that we can understand more of their consumption patterns. Before running ICA, we can use a preprocessing technique such as [`Scale`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html) to center the data to zero; removing the mean can help measure the variance of the data now from the origin (origin). This could still be done by keeping the mean, the only difference now is that the center is the mean itself, and not 0. The calculations are more simpler when you remove the mean as oppose to keeping it. Also, having the values centered at zero can capture the distributions when transforming the data.


- ICA1: Mostly includes frozen items (highest magnitude); customers tend to buys more grocery/frozen items and less detergents_paper/delicatessen/fresh/milk products.


- ICA2: Customer who buys more fresh products buys less milk/detergents_paper/delicatessen


- ICA3: Most significant is orders for delicatessens while decrease in the remaining products


- ICA4: Grocery has the highest magnitude (0.11); increase in grocery purchases is inversely related to detergents (decrease).


- ICA5: Customer purchases mostly milk/grocery/ will buy less slightly less fresh items and detergents (perhaps small market store)


- ICA6: Primarly as purchase orders for grocery increases, milk/fresh decrease  


In [ ]:
#Adjust the data to have center at the origin:
columns = data.columns
scale = preprocessing.scale(data)
df_scale = pd.DataFrame(scale, columns=columns)
#print data_scaled.mean()
#print data_scaled.std()

def doICA():
    ica = FastICA(n_components = 6, random_state=42)
    ica.fit(df_scale)
    return ica

# Print the independent components
ica = doICA()

In [ ]:
df_ica = pd.DataFrame(ica.components_,columns=columns, index=['1','2','3','4','5','6'])
df_ica.index.names = ['IC']
print "\nUnmixing Matrix:"
df_ica

In [ ]:
pl.figure(figsize=(20,20))
pl.figure(figsize = (11,5))
pl.title('Independent Component Analysis', fontsize=18)
sns.heatmap(df_ica, annot = True)

# Clustering

Using K Means clustering or Gaussian Mixed Models clustering, which implements expectation-maximization. Then sample elements from the clusters to understand their significance.

**5)** What are the advantages of using K Means clustering or Gaussian Mixture Models?

- In unsupervised learning, we are not predicting an outcome (no target label is given "y"); we are interested in how the features are related. We can group the data into "clusters" and two common techniques are K-means and Gaussian Mixture Models.


- K-mean is an iterative algorithm that is based on the idea that there exist a specific number of clusters or groups in the dataset and finds clusters that consists of similar characteristics that are more related to each other than in other groups. Each group in the data is distributed around a central point called the "centroid" which is the average of the cluster. K-means begins by initially guessing the K-number of clusters and then the algorithm randomly picks the centroid as the initial clusters of the data (not the correct centers for the first iteration). All of the points are assigned to the nearest centroid based on the Euclidian distance and is grouped into a cluster. The new centroid will be the mean of data points assigned to the corresponding cluster. This process is repeated until either the number of specified iterations is reached or until the clustering assignments do not change. Using K-Means in scikit-learn, the iteration will stop early if it converges before the maximum number of iterations is reached. As a note, incorrectly assigning the number of clusters for k can result in poor clustering performance. Next, each sample is assigned to the nearest centroid and then move the centroid to the center of the corresponding samples. 


- K- mean is a hard clustering algorithm, where the clusters to not overlap (i.e. the data point is either "blue" or "green". The advantages of using K-means is due to it's simplicity, speed, and scalability of larger numbers of data points. However, one weakness when using this method is that the initial centroids are placed randomly, which could result in a bad starting spot and the iteration could stop at an unlikely solution. Thus numerous iterations need to be run which can come at a cost of more time required to run K-means, however the trade-off is that more iteration that are conducted, the better the results will be.  


- Mixture models are a probabilistically way of doing soft clustering, where clusters may overlap. Gaussian Mixture Models (GMM) assumes that all of the data points in each cluster are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. GMM use a Gaussian distributions to find the most probable cluster that a point would belong to. The mixture model attempts to find unknown parameters such as the mean/covariance of the Gaussian distribution. 

- The different clustering algorithms and their usage can be found [`here`](http://scikit-learn.org/stable/modules/clustering.html)

**6)** Below is the code to visualize the clusters (Gaussian Mixture Model & K-Means)

In [ ]:
# Import clustering modules
from sklearn.cluster import KMeans
from sklearn.mixture import GMM

In [ ]:
# TODO: Reduce to Two dimensions using PCA to capture variation
reduced_data = PCA(n_components = 2).fit_transform(data)
df_rd = pd.DataFrame(reduced_data)
df_rd[:11]

- In order to evaluate the quality of the clustering, we need to find the optimal number of clusters. This can be done by computing the inertia, a method for comparing the performance of different k-means clusters (n_clusters). Inertia is calculated by taking the summation of the difference between every data point in each cluster and it's respected centroid. Inertia is accessible is calling the `inertia__` function, which computes with within cluster SSE. If the data points in the cluster are similar to it's centroid, the difference is small, resulting in a small inertia. Plotting the number of clusters vs the inertia yield a plot below. What is important from this plot is to observe the rate of change. We see that as the number of clusters increase, the lower the inertia will be. This is due to the samples being closer to their corresponding centroids. Basically, the idea behind this plot is to identify the value of k where the inertia begins to increase most rapidly, thus the optimal number of clusters to consider in this model is between 4-5. 

In [ ]:
def number_clusters():
    inertia = []
    delta_inertia = []
    for i in range(1,11):
        clustering = KMeans(n_clusters = i,
                    n_init=10,
                    max_iter=300,
                    random_state=1)
        clustering.fit(reduced_data)
        if inertia: 
            delta_inertia.append(inertia[-1] - clustering.inertia_)
        inertia.append(clustering.inertia_)
    
    pl.figure(figsize=(8,6))
    pl.plot([k for k in range(1,10)], delta_inertia, marker='o', color='g')
    pl.title('Optimal Number of Clusters', fontsize=18)
    pl.xlabel('Number of Clusters', fontsize=14)
    pl.ylabel('Rate of Change (Inertia)', fontsize=14)
    pl.show
    
number_clusters()

# Gaussian Mixture Model

In [ ]:
def cluster_plot_gmm():   
    n = 5
    # TODO: Implement clustering algorithm and fitting to the reduced data for visualization
    gmm = GMM(n_components=n, n_iter=300)
    clusters = gmm.fit(reduced_data)
    centroids = clusters.means_
    
    x_gmm = clusters.predict(reduced_data)

    # Which customers are in which cluster: K-Means
    for i in range(0,5):
        count = data[x_gmm==i].shape[0]
        print 'Gaussian Mixture Model:'
        print 'Cluster Number: %s' % (i+1)
        print 'Number of samples in cluster: %s' % count
        print data[x_gmm==i][list(data.columns[:-1])].mean()

    # Plot the decision boundary by building a mesh grid to populate a graph.
    x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
    y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
    hx = (x_max-x_min)/1000.
    hy = (y_max-y_min)/1000.
    xx, yy = np.meshgrid(np.arange(x_min, x_max, hx), np.arange(y_min, y_max, hy))

    # Obtain labels for each point in mesh. Use last trained model.
    Z = clusters.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    pl.figure(figsize=(8,6))
    Z = Z.reshape(xx.shape)
    pl.figure(1)
    pl.clf()
    pl.imshow(Z, interpolation='nearest',
                extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                cmap=pl.cm.Paired,
                aspect='auto', origin='lower')

    pl.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=5)
    pl.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=150, linewidths=4,
                color='w', zorder=10)
    pl.title('Clustering on the wholesale grocery dataset (PCA-reduced data)\n'
            'White X=Centroids; No. of Clusters:%s'%(5), fontsize = 14)
    pl.xlim(x_min, x_max)
    pl.ylim(y_min, y_max)
    pl.xticks(())
    pl.yticks(())

    # Annotate Plot:
    labels = ['{0}'.format(i) for i in range(1,6)]
    for label, x, y in zip(labels, centroids[:,0], centroids[:,1]):
         pl.annotate(label, xy = (x, y), xytext = (-20, 20),
         textcoords = 'offset points', size=14, ha = 'right', va = 'bottom',
         bbox = dict(boxstyle="round4,pad=0.3", fc="cyan", ec="b", lw=2),
         arrowprops = dict(arrowstyle = 'simple', connectionstyle = 'arc3,rad=-0.3'))
    pl.show()
    
    data['cluster'] = clusters.predict(reduced_data)
    results = pd.DataFrame(data=labels, columns=['cluster']) 
    data_grouped = data.groupby('cluster').mean()
    
    # Histogram
    ax = data_grouped.plot(kind='bar',legend=True, 
                    figsize=(10,6), alpha=.5, title="Customer Breakdown")
    ax.set_xlabel("Clusters",fontsize=16)
    ax.set_ylabel("Total Monetary Units",fontsize=16)
    ax.set_title(ax.get_title(), fontsize=20)
    pl.show()

cluster_plot_gmm()

# K-Means

In [ ]:
def cluster_plot_kmeans():  
    num_clusters = 5
    clusters = KMeans(n_clusters=num_clusters, n_init=100)
    clusters.fit(reduced_data)
    centroids = clusters.cluster_centers_
    
    x_km = clusters.predict(reduced_data)

    # Which customers are in which cluster: K-Means
    for i in range(0,5):
        count = data[x_km==i].shape[0]
        print 'Kmeans:'
        print 'Cluster Number: %s' % (i+1)
        print 'Number of samples in cluster: %s' % count
        print data[x_km==i][list(data.columns[:-1])].mean()

    # Plot the decision boundary by building a mesh grid to populate a graph.
    x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
    y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
    hx = (x_max-x_min)/1000.
    hy = (y_max-y_min)/1000.
    xx, yy = np.meshgrid(np.arange(x_min, x_max, hx), np.arange(y_min, y_max, hy))

    # Obtain labels for each point in mesh. Use last trained model.
    Z = clusters.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    pl.figure(figsize=(8,6))
    Z = Z.reshape(xx.shape)
    pl.figure(1)
    pl.clf()
    pl.imshow(Z, interpolation='nearest',
                extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                cmap=pl.cm.Paired,
                aspect='auto', origin='lower')

    pl.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=5)
    pl.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=150, linewidths=4,
                color='w', zorder=10)
    pl.title('Clustering on the wholesale grocery dataset (PCA-reduced data)\n'
            'White X=Centroids; No. of Clusters:%s'%(5), fontsize = 14)
    pl.xlim(x_min, x_max)
    pl.ylim(y_min, y_max)
    pl.xticks(())
    pl.yticks(())

    # Annotate Plot:
    labels = ['{0}'.format(i) for i in range(1,6)]
    for label, x, y in zip(labels, centroids[:,0], centroids[:,1]):
         pl.annotate(label, xy = (x, y), xytext = (-20, 20),
         textcoords = 'offset points', size=14, ha = 'right', va = 'bottom',
         bbox = dict(boxstyle="round4,pad=0.3", fc="cyan", ec="b", lw=2),
         arrowprops = dict(arrowstyle = 'simple', connectionstyle = 'arc3,rad=-0.3'))
    pl.show()
    
    print "K-Means:"
    print "Number of Customers per Cluster:"
    print data['cluster'].value_counts()

    data['cluster'] = clusters.labels_
    results = pd.DataFrame(data=labels, columns=['cluster']) 
    data_grouped = data.groupby('cluster').mean()
    
    # Histogram
    ax = data_grouped.plot(kind='bar',legend=True, 
                    figsize=(10,6), alpha=.5, title="Customer Breakdown")
    ax.set_xlabel("Clusters",fontsize=16)
    ax.set_ylabel("Total Monetary Units",fontsize=16)
    ax.set_title(ax.get_title(), fontsize=20)
    pl.show()

cluster_plot_kmeans()

**7**) What are the central objects in each cluster? Describe them as customers.

- The central object of each cluster is in reference to the above Gaussian Mixture Model (GMM) and K-Means cluster plot and histogram. For the sake of this discussion, we will explore the results from the GMM. As a note on the histogram, the cluster labeling begins at 0; this refers to cluster 1 and so on. What we find in the first cluster is a higher precentage of fresh, milk, grocery, and frozen products but with lowest consumption among the remaining clusters. To the left of cluster 1 is the second cluster (green) which contains 133 samples. The fresh items are the most significant, which reflects the results discussed earlier with `fresh` having the highest magnitude in the first PC, thus more influential in the results. The first two clusters indiciate this customer base to be more smaller familiy run corner store or convience store with lower consumption patterns. The third cluster shows larger consumption patterns which would classify this group as the larger grocery stores such as Whole Foods or Trader Joes. Only 17 stores fall into this cluster, which is just a small fraciton of the total customer base. The fourth cluster has a total of 38 customers and the most significant item in this cluster is a high consumption of fresh products. Consumption for Milk, grocery, and frozen items drops off by nearly 75%. Perhaps this cluster represents the patterns associated with a restaurant that has a high density of fresh products such as a deli. The fifth cluster consists of a customer base that would most likely represent a smaller neighborhood grocery store with the bulk of the products purchased being fresh/milk/grocery items. 

# Conclusions:


**8**) Which of these techniques did you feel gave you the most insight into the data?

- Applying Principal Component Analysis was one of the most useful techniques as it provides insight into the data. We were able to reduce the high dimensional dataset to a lower dimensional space which allowed the data to be visualized on a two dimensional scatter plot. The data was projected onto the new principal component axes and each datapoint in the plot represents a single customer (store). Additionally, PCA is a feature extractor, thus we are able to identify which attributes are most significant in each principal component. After applying PCA, the second most useful method is then implementing GMM to the reduced dataset in order to group the customers into respective market segments, which can enhance better decision making.

**9**) How would you use that technique to help the company design new experiments?

- As we have classified the wholesale customers into different markets, applying the A/B test would help the company with making any changes to the business model. A/B testing, often called split testing, is widely used when creating web pages by comparing two versions of a web page and see which one performs better. Thus, applying A/B testing to this dataset will allow the company to make optimal decisions by trying out possible changes and seeing what performs better. For an example, we may want to test various pricing options for products to monitor how the change in price affects purchase rate for a particular market. We can split the cluster into two groups with the goal of determining if the rate of success is higher in one group (A) than the other (B). One way to do this is determine the p-value to get an idea of how confident we are that the two groups have different chances of success. Using the common baseline of "statistically significant" at 0.05, a p-value lower than 0.05 means the two groups are different. But the estimated improvement in success rate would give a better idea of how much better/worse any changes might be compared to initial prices. There are numerous instances on what could be analyzed, but this example would just give an idea on how we can make optimal business decisions by applying any new changes to how the company operates. 

**10**) How would you use that data to help you predict future customer needs?

- Once we have defined the customer base, we can then target particular customers by performing different tests in order to enhance profit and growth of the company. After applying PCA and GMM, we have grouped the data into clusters and now the company can utilize a supervised machine learning model, either regression or classification, to predict which cluster the new customer is likely to fall in. For instance, we could extract the customers and their corresponding consumption elements (Fresh, frozen, etc.) from the Gaussian Mixture Model and use then use their respected cluster as the label to use the classification learning algorithm. This would then allow the company to make prediction on future customers and their purchase orders. Another suggestion would be to use regression on the clusters that were identified in order to predict the current wholesale customers in addition to any new customers.